In [ ]:
# Install  Packets

import sys
!{sys.executable} -m pip install lightfm
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install surprise
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install scikit-surprise 
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install google.colab
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install plotly
!{sys.executable} -m pip install fastparquet
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install Pylance
!{sys.executable} -m pip install fuzzywuzzy
#Only Once
## https://academy.rapidminer.com/learn/article/recommendation-engine-youre-going-to-love-this-movie


In [ ]:
# To store the data
import pandas as pd

# To do linear algebra
import numpy as np

# To create plots
import matplotlib.pyplot as plt
import seaborn as sns

# To create interactive plots
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# To shift lists
from collections import deque

# To compute similarities between vectors
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# To use recommender systems
import surprise as sp
from surprise.model_selection import cross_validate

# To create deep learning models
from keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout
from keras.models import Model

# To create sparse matrices
from scipy.sparse import coo_matrix

# To light fm
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

# To stack sparse matrices
from scipy.sparse import vstack
from scipy.sparse import csr_matrix



The ratings per movie as well as the ratings per user both have nearly a perfect **exponential decay**. Only very few 
movies/users have many ratings. 

***
## <a id=8>8. Filter Sparse Movies And Users</a>

To reduce the dimensionality of the dataset I am filtering rarely rated movies and rarely rating users out.

In [ ]:
#Als Nächstes sollen alle Pfade mit variablen belegt werden, dass macht das austauschen einfacher.

movie_tile_File = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/movie_titles.csv'
movie_tile_File_new = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/movie_titles_new.csv'
combined_data_1 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_1.txt'
combined_data_2 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_2.txt'
combined_data_3 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_3.txt'
combined_data_4 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_4.txt'
new_Combined = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/Cobined_data_new_v1.csv'
netflix_rating_Combined = 'C:/Users/jsbreite\OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/netflix_data.csv'
parquet_combined_data = 'C:/Users/jsbreite\OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/data_Comb.zip'

In [ ]:
#Load Data into viables

combined_data_1_raw = pd.read_csv(combined_data_1, header=None, names=['Cust_Id', 'Rating'], usecols=[0, 1])
combined_data_2_raw = pd.read_csv(combined_data_2, header=None, names=['Cust_Id', 'Rating'], usecols=[0, 1])
combined_data_3_raw = pd.read_csv(combined_data_3, header=None, names=['Cust_Id', 'Rating'], usecols=[0, 1])
combined_data_4_raw = pd.read_csv(combined_data_4, header=None, names=['Cust_Id', 'Rating'], usecols=[0, 1])

In [ ]:
## Combined_DATA_1
# Find empty rows to slice dataframe for each movie
tmp_movies = combined_data_1_raw[combined_data_1_raw['Rating'].isna()]['Cust_Id'].reset_index()  
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]

# Shift the movie_indices by one to get start and endpoints of all movies
shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)


# Gather all dataframes
user_data = []

# Iterate over all movies
for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    # Check if it is the last movie in the file
    if df_id_1<df_id_2:
        tmp_df = combined_data_1_raw.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = combined_data_1_raw.loc[df_id_1+1:].copy()
        
    # Create movie_id column
    tmp_df['Movie_Id'] = movie_id
    
    # Append dataframe to list
    user_data.append(tmp_df)

# Combine all dataframes
df_1 = pd.concat(user_data)
del user_data, combined_data_1_raw, combined_data_1, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id
# print('Shape User-Ratings:\t{}'.format(df_1.shape))

## Combined_DATA_2
# Find empty rows to slice dataframe for each movie
tmp_movies = combined_data_2_raw[combined_data_2_raw['Rating'].isna()]['Cust_Id'].reset_index()  ##---> Nachvollziehen
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]

# Shift the movie_indices by one to get start and endpoints of all movies
shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)


# Gather all dataframes
user_data = []

# Iterate over all movies
for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    # Check if it is the last movie in the file
    if df_id_1<df_id_2:
        tmp_df = combined_data_2_raw.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = combined_data_2_raw.loc[df_id_1+1:].copy()
        
    # Create movie_id column
    tmp_df['Movie_Id'] = movie_id
    
    # Append dataframe to list
    user_data.append(tmp_df)

# Combine all dataframes
df_2 = pd.concat(user_data)
del user_data, combined_data_2_raw, combined_data_2, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id
# print('Shape User-Ratings:\t{}'.format(df_2.shape))

## Combined_DATA_3
# Find empty rows to slice dataframe for each movie
tmp_movies = combined_data_3_raw[combined_data_3_raw['Rating'].isna()]['Cust_Id'].reset_index()  ##---> Nachvollziehen
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]

# Shift the movie_indices by one to get start and endpoints of all movies
shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)


# Gather all dataframes
user_data = []

# Iterate over all movies
for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    # Check if it is the last movie in the file
    if df_id_1<df_id_2:
        tmp_df = combined_data_3_raw.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = combined_data_3_raw.loc[df_id_1+1:].copy()
        
    # Create movie_id column
    tmp_df['Movie_Id'] = movie_id
    
    # Append dataframe to list
    user_data.append(tmp_df)

# Combine all dataframes
df_3 = pd.concat(user_data)
del user_data, combined_data_3_raw, combined_data_3, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id
# print('Shape User-Ratings:\t{}'.format(df_3.shape))

## Combined_DATA_4
# Find empty rows to slice dataframe for each movie
tmp_movies = combined_data_4_raw[combined_data_4_raw['Rating'].isna()]['Cust_Id'].reset_index()  ##---> Nachvollziehen
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]

# Shift the movie_indices by one to get start and endpoints of all movies
shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)


# Gather all dataframes
user_data = []

# Iterate over all movies
for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    # Check if it is the last movie in the file
    if df_id_1<df_id_2:
        tmp_df = combined_data_4_raw.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = combined_data_4_raw.loc[df_id_1+1:].copy()
        
    # Create movie_id column
    tmp_df['Movie_Id'] = movie_id
    
    # Append dataframe to list
    user_data.append(tmp_df)

# Combine all dataframes
df_4 = pd.concat(user_data)
del user_data, combined_data_4_raw, combined_data_4, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id
print('Shape User-Ratings:\t{}'.format(df_4.shape))

#Zusammenfügen der aller Daten in einer Variable
data = [df_1, df_2,df_3,df_4]
df = pd.concat(data)
del df_1, df_2, df_3, df_4
print(df.tail(5))
print(df.head(5))

The ratings per movie as well as the ratings per user both have nearly a perfect **exponential decay**. Only very few 
movies/users have many ratings. 

***
## <a id=8>8. Filter Sparse Movies And Users</a>

To reduce the dimensionality of the dataset I am filtering rarely rated movies and rarely rating users out.

In [ ]:
# Filter sparse movies
min_movie_ratings = 20000
filter_movies = (df['Movie_Id'].value_counts()>min_movie_ratings)
filter_movies = filter_movies[filter_movies].index.tolist()

# Filter sparse users
min_user_ratings = 200
filter_users = (df['Cust_Id'].value_counts()>min_user_ratings)
filter_users = filter_users[filter_users].index.tolist()

# Actual filtering
df_filterd = df[(df['Movie_Id'].isin(filter_movies)) & (df['Cust_Id'].isin(filter_users))]
del filter_movies, filter_users, min_movie_ratings, min_user_ratings
print('Shape User-Ratings unfiltered:\t{}'.format(df.shape))
print('Shape User-Ratings filtered:\t{}'.format(df_filterd.shape))

In [76]:
num_users = len(df.Cust_Id.unique())
num_items = len(df.Movie_Id.unique())
print('There are {} unique users and {} unique movies in this data set'.format(num_users, num_items))


# get count
df_ratings_cnt_tmp = pd.DataFrame(df.groupby('Rating').size(), columns=['count'])
df_ratings_cnt_tmp

# there are a lot more counts in rating of zero
total_cnt = num_users * num_items
rating_zero_cnt = total_cnt - df.shape[0]

# append counts of zero rating to df_ratings_cnt
df_ratings_cnt = df_ratings_cnt_tmp.append(
    pd.DataFrame({'count': rating_zero_cnt}, index=[0.0]),
    verify_integrity=True,
).sort_index()
df_ratings_cnt

# add log count
df_ratings_cnt['log_count'] = np.log(df_ratings_cnt['count'])
df_ratings_cnt

# get rating frequency
df_movies_cnt = pd.DataFrame(df.groupby('Movie_Id').size(), columns=['count'])
df_movies_cnt.head()

## Andere Art zu Filtern

# filter data
popularity_thres = 50
popular_movies = list(set(df_movies_cnt.query('count >= @popularity_thres').index))
df_ratings_drop_movies = df[df.Movie_Id.isin(popular_movies)]
print('shape of original ratings data: ', df.shape)
print('shape of ratings data after dropping unpopular movies: ', df_ratings_drop_movies.shape)

# get number of ratings given by every user
df_users_cnt = pd.DataFrame(df_ratings_drop_movies.groupby('Movie_Id').size(), columns=['count'])
df_users_cnt.head()

There are 480189 unique users and 17770 unique movies in this data set


C:\Users\jsbreite\AppData\Local\Temp\ipykernel_22444\821935844.py:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



shape of original ratings data:  (100480507, 3)
shape of ratings data after dropping unpopular movies:  (100478440, 3)


In [78]:

# filter data # in 23
ratings_thres = 50
active_users = list(set(df_users_cnt.query('count >= @ratings_thres').index))
df_ratings_drop_users = df_ratings_drop_movies[df_ratings_drop_movies.Cust_Id.isin(active_users)]
print('shape of original ratings data: ', df.shape)
print('shape of ratings data after dropping both unpopular movies and inactive users: ', df_ratings_drop_users.shape)

shape of original ratings data:  (100480507, 3)
shape of ratings data after dropping both unpopular movies and inactive users:  (0, 3)


In [ ]:
print(df_filterd.head(5))

### https://github.com/KevinLiao159/MyDataSciencePortfolio/blob/master/movie_recommender/movie_recommendation_using_KNN.ipynb

In [83]:
movie_titles_Columns = pd.read_csv(movie_tile_File, 
                           encoding = 'ISO-8859-1', 
                           engine = 'python',
                           delimiter =',',
                           header = None, 
                           on_bad_lines= 'skip', #Wird gebraucht um Fehlerhafte Title zu behen.

                           names = ['Movie_Id', 'Year', 'title'])

movie_titles = movie_titles_Columns.drop(columns='Year')

del movie_titles_Columns

In [84]:
print(movie_titles.head(5))

   Movie_Id                         title
0         1               Dinosaur Planet
1         2    Isle of Man TT 2004 Review
2         3                     Character
3         4  Paula Abdul's Get Up & Dance
4         5      The Rise and Fall of ECW


In [85]:
# pivot and create movie-user matrix
movie_user_mat = df_ratings_drop_users.pivot(index='Movie_Id', columns='Cust_Id', values='Rating').fillna(0)
# create mapper from movie title to index
movie_to_idx = {
    movie: i for i, movie in 
    enumerate(list(movie_titles.set_index('Movie_Id').loc[movie_user_mat.index].title))
}
# transform matrix to scipy sparse matrix
movie_user_mat_sparse = csr_matrix(movie_user_mat.values)

In [88]:
%env JOBLIB_TEMP_FOLDER=/tmp
# define model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
# fit
model_knn.fit(movie_user_mat_sparse)

env: JOBLIB_TEMP_FOLDER=/tmp


ValueError: Found array with 0 sample(s) (shape=(0, 0)) while a minimum of 1 is required.